In [1]:
from LoadingBMWDataset import ObjectDetectionDataset
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json
import os
from PIL import Image
import shutil

In [2]:
def convert_annotations_to_yolo_format(img_dir, annotation_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    # Iterating over each annotation file in the annotation directory, and convert the annotations to YOLO format
    for annotation_file in os.listdir(annotation_dir):
        if annotation_file.endswith('.json'):
            json_path = os.path.join(annotation_dir, annotation_file)
            with open(json_path) as f:
                annotations = json.load(f)
            
            # Deriving the corresponding image file path and load it to get dimensions
            img_file = annotation_file.replace('.json', '.jpg')
            img_path = os.path.join(img_dir, img_file)
            with Image.open(img_path) as img:
                img_width, img_height = img.size
            
            # Converting annotations to YOLO format
            yolo_annotations = []
            for annot in annotations:
                class_id = annot['ObjectClassId'] - 1  # Assuming class IDs start from 1, adjust if necessary
                x_center = ((annot['Right'] + annot['Left']) / 2) / img_width
                y_center = ((annot['Bottom'] + annot['Top']) / 2) / img_height
                width = (annot['Right'] - annot['Left']) / img_width
                height = (annot['Bottom'] - annot['Top']) / img_height
                yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")
            
            # Saving converted annotations to TXT file
            txt_path = os.path.join(output_dir, annotation_file.replace('.json', '.txt'))
            with open(txt_path, 'w') as f:
                f.write('\n'.join(yolo_annotations))


In [3]:
img_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images'
annotation_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/json'
output_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/yolo'

convert_annotations_to_yolo_format(img_dir, annotation_dir, output_dir)

In [5]:
from sklearn.model_selection import train_test_split
import glob

annotations_txt_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/yolo'
img_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images'

# Listing all .txt annotation files
annotation_files = glob.glob(os.path.join(annotations_txt_dir, '*.txt'))

# Extracting corresponding image file names from annotation file names
img_files = [os.path.join(img_dir, os.path.basename(f).replace('.txt', '.jpg')) for f in annotation_files]

# Splitting the dataset into training and validation
train_img_files, val_img_files, train_annotation_files, val_annotation_files = train_test_split(img_files, annotation_files, test_size=0.2, random_state=42)
